In [1]:
import os
import io
import tarfile
import pandas as pd
import numpy as np

In [2]:
from xlrd.compdoc import CompDocError
import xlrd

In [3]:
# Suppress warning for xlrd
import warnings
# Filter all warning that start with "WARNING"
warnings.filterwarnings('ignore')

In [5]:
# pass over all files and build summary
file = '/Users/dirkocoetsee/Downloads/fuse-binaries-dec2014.tar.gz'
# Get list of files in tar archive
tar = tarfile.open(file)
all_members = tar.getmembers()

In [6]:
len(all_members)

249377

In [7]:
tar.extractfile(all_members[1])
all_members[1].name

'cc-binaries/25401149-02fa-402e-94f4-c660bcf386b6'

In [8]:
for i, member in enumerate(all_members):
    if i > 10:
        break
    if member.name == 'cc-binaries':
        continue
    print(member.name)
    buffer = tar.extractfile(member)
    # Read the excel file in the buffer
    try:
        df = pd.read_excel(io.BytesIO(buffer.read()))
    except CompDocError:
        print('  CompDocError')
        continue
    print('  ', df.shape)

cc-binaries/25401149-02fa-402e-94f4-c660bcf386b6
   (24, 62)
cc-binaries/70b81cf4-d32d-4de9-a40a-5dc044eaa14b
   (5356, 7)
cc-binaries/2b00234b-b53b-4516-ade5-4862a42f33a7
WARNING *** file size (10243) not 512 + multiple of sector size (512)
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 0 
  CompDocError
cc-binaries/0ba21114-298f-4787-840b-2bf313f7e46a
   (72, 3)
cc-binaries/90d8044f-ab27-4d1d-b97d-b6097bf79f79
WARNING *** file size (20483) not 512 + multiple of sector size (512)
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 
   20  4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 0 
  CompDocError
cc-binaries/694888d7-1470-482e-b770-e006622d7f25
WARNING *** file size (12802) not 512 + multiple of sector size (512)
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 
   20  4 4 3 2 0 
  CompDocError
cc-binaries/fe36cad9-4ffd-439a-b4ff-eefed108c1e6
   (74, 3)
cc-binaries/617b82c1-f2c8-4cd4-98aa-a3d1d03cbb8f
WAR

In [9]:
vocab = {}
words = {}
for i, member in enumerate(all_members):
    if i > 1000:
        break
    #print(member.name)
    buffer = tar.extractfile(member)
    try:
        df = pd.read_excel(io.BytesIO(buffer.read()), header=None, index_col=None)
    except CompDocError:
        continue
    except AttributeError:
        continue
    except IndexError:
        continue
    except:
        continue
    # Iterate over all cells
    vals = {v for v in df.values.flatten()}
    # Update vocab
    vocab_size = len(vocab)
    # Add new words to vocab
    vocab.update({v: vocab_size+i for i, v in enumerate(val for val in vals if val not in vocab)})
    # Update words
    words[member.name] = {vocab[v] for v in vals}
    print(f'{i} /  {len(all_members)}', member.name, '  ', df.shape, len(vals), len(vocab))


1 /  249377 cc-binaries/25401149-02fa-402e-94f4-c660bcf386b6    (25, 62) 885 885
2 /  249377 cc-binaries/70b81cf4-d32d-4de9-a40a-5dc044eaa14b    (5357, 7) 15246 16109
WARNING *** file size (10243) not 512 + multiple of sector size (512)
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 0 
4 /  249377 cc-binaries/0ba21114-298f-4787-840b-2bf313f7e46a    (73, 3) 162 16266
WARNING *** file size (20483) not 512 + multiple of sector size (512)
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 
   20  4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 0 
WARNING *** file size (12802) not 512 + multiple of sector size (512)
_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 
   20  4 4 3 2 0 
7 /  249377 cc-binaries/fe36cad9-4ffd-439a-b4ff-eefed108c1e6    (75, 3) 165 16382
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
8 /  249377 cc-binaries/617b82c1-f2c8-4cd4-98aa-a3d1d03cbb8f    (78, 8) 200 16558
9

In [10]:
len(words)

643

In [11]:
x = np.zeros((len(words), len(vocab) + 0))
for i, (k, v) in enumerate(words.items()):
    x[i, list(v)] = 1

In [ ]:
# Save to disk
#np.savez_compressed('data.npz', x=x, words=list(words.keys()))
# Tooslow..

In [ ]:
#!ls -lah data.npz

In [12]:
max(v for k, v in vocab.items()), len(vocab)

(814812, 814813)

In [ ]:
kass

In [ ]:
x.shape

In [ ]:
# Drop unique columns
lim = 10000
cols = np.where(x[:lim, :].sum(axis=0) > 1)[0]
xd = x[:lim, cols]

In [ ]:
xd.shape

In [ ]:
# First lower the dimension by randomly selecting a subset of the features
dim =
xr = x[:, np.random.choice(x.shape[1], 1000, replace=False)]

In [ ]:
x[:10, :].sum(axis=1)[:10]

In [ ]:
import faiss
D = x.shape[1]
index = faiss.IndexFlatL2(D)  # L2 or cosine similarity
index.add(x)  # Add dataset

In [ ]:
k = 20
_, indices = index.search(x[:100, :], k)  # Perform search

top = indices[:, 1:]

In [ ]:
top.shape

In [13]:
r = np.dot(x, x.T)

In [14]:
# Set diagonal to -1
rr = r.copy()
np.fill_diagonal(rr, -1)

In [15]:
# Get indices of top 20 per row (excluding self)
top = np.argsort(rr, axis=1)[:, -20:]

In [16]:
# Get r[i, j] for all i, j in top per row
t = np.zeros((len(words), 20, 20))
for i in range(len(words)):
    for j in range(20):
        for k in range(20):
            t[i, j, k] = r[top[i, j], top[i, k]]

In [17]:
# Ravel along last two axes, leaving first axis
ind = np.unravel_index(np.argsort(t.reshape((len(words), -1)), axis=1)[:, :10], (20, 20))
ind[0].shape, ind[1].shape

((643, 10), (643, 10))

In [18]:
ind[0]

array([[ 1,  3,  2, ...,  0,  2, 16],
       [19,  0, 19, ..., 19, 19, 19],
       [ 3,  0,  0, ..., 11, 11,  1],
       ...,
       [ 1,  1,  8, ...,  9,  4, 12],
       [18,  5,  3, ..., 18, 18, 13],
       [ 2,  2,  1, ...,  1,  4,  9]])

In [19]:
ind[0][:, 0].shape

(643,)

In [20]:
# Get all indices from top that correspond to ind[0]
np.take_along_axis(top, ind[0], axis=1)

array([[156, 162, 323, ..., 503, 323, 423],
       [439, 603, 439, ..., 439, 439, 439],
       [307, 339, 339, ..., 314, 314,  49],
       ...,
       [237, 237, 434, ..., 233, 453, 614],
       [291,   6, 281, ..., 291, 291, 387],
       [217, 217, 133, ..., 133, 428, 321]])

In [21]:
i0 = np.take_along_axis(top, ind[0], axis=1)
i1 = np.take_along_axis(top, ind[1], axis=1)
# Map back to filenames
index_name_map = {i: k for i, (k, v) in enumerate(words.items())}

# Get filenames for the first tripple
f1 = (index_name_map[0])
f2 = (index_name_map[i0[0][0]])
f3 = (index_name_map[i1[0][0]])
print(f1, f2, f3)

cc-binaries/25401149-02fa-402e-94f4-c660bcf386b6 cc-binaries/a23b1b5e-7d23-4981-8e34-0050052e5b54 cc-binaries/7940585a-e9be-4ff4-bd78-dc695d9645cf


In [22]:
i0.shape

(643, 10)

In [23]:
# Get score for each tripple
tripples = []
sums = r.sum(axis=1)
for ir in range(len(words)):
    for ic1, ic2 in zip(i0[ir, :], i1[ir, :]):
        alpha1 = r[ir, ic1] / sums[ir]
        alpha2 = r[ir, ic2] / sums[ir]
        score = alpha1 + alpha2 - r[ic1, ic2] / max(sums[ic1], sums[ic2])
        if ir < 2:
            print(ir, ic1, ic2, f'{score:.3f}', f'{alpha1:.3f}', f'{alpha2:.3f}')
        tripples.append((ir, ic1.item(), ic2.item(), score, alpha1, alpha2))
len(tripples)


0 156 162 0.019 0.011 0.011
0 162 156 0.019 0.011 0.011
0 323 591 0.023 0.011 0.014
0 591 323 0.023 0.014 0.011
0 503 162 0.020 0.011 0.011
0 162 503 0.020 0.011 0.011
0 323 503 0.019 0.011 0.011
0 503 323 0.019 0.011 0.011
0 323 306 0.031 0.011 0.022
0 423 323 0.031 0.022 0.011
1 439 603 0.006 0.005 0.003
1 603 439 0.006 0.003 0.005
1 439 255 0.007 0.005 0.003
1 439 267 0.007 0.005 0.003
1 290 439 0.007 0.003 0.005
1 439 31 0.007 0.005 0.003
1 255 439 0.007 0.003 0.005
1 439 290 0.007 0.005 0.003
1 439 589 0.007 0.005 0.003
1 439 47 0.007 0.005 0.003


6430

In [24]:
# Get value from np.int64
ic1.item()

321

In [25]:
# Filter out tripples with low alpha1 or alpha2
threshold = 0.02
ftripples = [t for t in tripples if t[4] > threshold and t[5] > threshold]
print(len(tripples), len(ftripples))

6430 21


In [26]:
for i, j, k, s, a1, a2 in ftripples[:10]:
    print(i, j, k, f'{s:.3f}', f'{a1:.3f}', f'{a2:.3f}')

100 199 589 0.069 0.040 0.029
100 199 323 0.068 0.040 0.028
100 199 312 0.063 0.040 0.023
100 199 255 0.062 0.040 0.022
100 199 522 0.062 0.040 0.021
167 324 267 0.026 0.023 0.021
167 324 263 0.025 0.023 0.021
167 324 20 0.025 0.023 0.021
167 324 94 0.025 0.023 0.021
167 232 324 0.025 0.021 0.023


In [27]:
# Load and display
for if1, if2, if3, score, _, _ in sorted(ftripples, key=lambda x: x[3], reverse=True)[:10]:
    f1 = (index_name_map[if1])
    f2 = (index_name_map[if2])
    f3 = (index_name_map[if3])
    for i, f in enumerate([f1, f2, f3]):
        buffer = tar.extractfile(f)
        df = pd.read_excel(io.BytesIO(buffer.read()))
        print(f'File {i} ({score}): {f}')
        print()
        # Display as html
        display(df)
        print()


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
File 0 (0.06922171210593919): cc-binaries/cb85c754-745c-4593-950e-fb0e986d0fe1



,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,NaN,U.S. Census Bureau
1,NaN,NaN,Source: Manufactured Homes Survey
2,NaN,NaN,Total Homes: U.S. Total
3,NaN,NaN,Not Seasonally Adjusted Total Shipments [Thous...
4,NaN,NaN,Period: 1959 to 2013
...,...,...,...
663,Aug-2013,5.9,NaN
664,Sep-2013,5.5,NaN
665,Oct-2013,6.1,NaN
666,Nov-2013,4.9,NaN



WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
File 1 (0.06922171210593919): cc-binaries/8b104259-c4e8-45b4-8b39-013edf5f79a5



,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,NaN,U.S. Census Bureau
1,NaN,NaN,Source: U.S. International Trade in Goods and ...
2,NaN,NaN,Balance of Payment Goods and Services: United ...
3,NaN,NaN,Seasonally Adjusted Balance [Millions of Dollars]
4,NaN,NaN,Period: 1992 to 2013
...,...,...,...
267,Aug-2013,-39515,NaN
268,Sep-2013,-42263,NaN
269,Oct-2013,-39083,NaN
270,Nov-2013,-35972,NaN



File 2 (0.06922171210593919): cc-binaries/3f1d5c77-e12b-49d6-a853-88d695d5169a



...

,LABEL,UPC,ARTIST,TITLE,RELEASEDATE,PRODUCTTYPE,GENRE,PRICE,RETURNABLE,STATUS
0,Fervent Records,679143004628,Barlowgirl,Barlowgirl,2005-06-19,Compact Disc,Rock,13.99,Y,Available
1,Word Records,80688856021,Everfound,Everfound,2013-07-16,Compact Disc,Rock,11.99,Y,Available
2,Brash Music,881410008523,Gungor,"Creation Liturgy, A",2012-10-16,Compact Disc,Alt,13.99,Y,Available
3,Warner Bros Rec,93624847229,Robert Randolph And The Family,Unclassified,2003-08-05,Compact Disc,Rock,13.99,Y,Available
4,Curb Records,715187915725,Plumb,Beautiful History: A Hits Collection,2009-11-03,Compact Disc,Rock,13.99,Y,Available
5,Inter-Company,854132001851,The Color Morale,We All Have Demons,2010-09-14,Compact Disc,Rock,12.98,Y,Available
6,Atlantic-Elektra,16861759926,The Devil Wears Prada,8:18,2013-11-08,Compact Disc,Rock,13.99,Y,Available
7,Wea-Ada,856658003548,Worthwhile,Carry On Kid,2013-08-13,Compact Disc,Rock,13.98,Y,Available
8,Wea-Ada,850537004503,The Color Morale,Know Hope,2013-04-23,Compact Disc,Rock,12.98,Y,Available
9,Inter-Company,854132001417,The Devil Wears Prada,Plagues (Deluxe Edition),2010-10-05,Compact Disc,Rock,13.98,Y,Available



File 1 (0.05504587155963303): cc-binaries/253e3a7a-012e-4bb7-b5be-d458bb4fe936



,Data Source,World Development Indicators,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Country Name,Country Code,Indicator Name,Indicator Code,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,...,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0
2,Aruba,ABW,Income share held by second 20%,SI.DST.02ND.20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aruba,ABW,Income share held by third 20%,SI.DST.03RD.20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aruba,ABW,Income share held by fourth 20%,SI.DST.04TH.20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4533,Zimbabwe,ZWE,Poverty headcount ratio at national poverty li...,SI.POV.NAHC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.3,NaN,NaN
4534,Zimbabwe,ZWE,Poverty gap at rural poverty line (%),SI.POV.RUGP,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.8,NaN,NaN
4535,Zimbabwe,ZWE,Poverty headcount ratio at rural poverty line ...,SI.POV.RUHC,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.3,NaN,NaN
4536,Zimbabwe,ZWE,Poverty gap at urban poverty line (%),SI.POV.URGP,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5,NaN,NaN



File 2 (0.05504587155963303): cc-binaries/6b7b5ad5-5482-43e1-b58c-1beb6b5db059



,LABEL,UPC,ARTIST,TITLE,RELEASEDATE,PRODUCTTYPE,GENRE,PRICE,RETURNABLE,STATUS
0,Warner Alliance Atlantic (Word,75678323126,Tracy-Dawn,Poetic Aftermath,2002-04-01,Compact Disc,Na,16.99,N,To Be Retired


In [ ]:
# For each slice along axis=0, get the axis=1 and axis=2 combination with the lowest value
for i in range(len(words)):
    t[i, :, :]

In [ ]:
t.shape